# Alura Store 
Análise Data-Driven: Recomendação baseada em faturamento, desempenho geográfico e satisfação do cliente

## 1. Contexto do Problema  

 Ajudar o Senhor João a decidir qual loja da sua rede Alura Store vender para iniciar um novo empreendimento.

- Objetivo: Identificar a loja menos eficiente para venda.  
- Dados disponíveis: Vendas em 2023, avaliações, localização. 

### 1.1 Instalando e importação das bibliotecas utilizadas

Utilizaremos diversas ferramentas para esse trabalho, _pandas_ para acessar as informações, _seaborn_ e _matplotlib_ para a vizualização dos dados e _folium_ para a análise geográfica. 

In [ ]:
!pip install requirements

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns
import folium
from folium.plugins import HeatMap

### 1.2 Importação dos dados

Foi repassado os seguintes dados referentes a cada loja. Com informação das vendas no período de 2021 a 


In [ ]:
# Carregar e consolidar os dados
loja1 = pd.read_csv('data/loja_1.csv')
loja2 = pd.read_csv('data/loja_2.csv')
loja3 = pd.read_csv('data/loja_3.csv')
loja4 = pd.read_csv('data/loja_4.csv')


### 1.3 Unificação dos dados

Para facilitar as análises a seguir, utilizarei uma abordagem de unificar cada _data frame_, para assim facilitar a leitura e evitar repetição de código.

In [ ]:
# Adicionando uma coluna 'Loja' em cada data frame
loja1['Loja'] = 'Loja 1'
loja2['Loja'] = 'Loja 2'
loja3['Loja'] = 'Loja 3'
loja4['Loja'] = 'Loja 4'


# Criando o único data frame
lojas = [loja1, loja2, loja3, loja4]

df = pd.concat(lojas, ignore_index=True)

df.head()

### Estilização dos gráficos
Explicação...

In [ ]:
sns.set_style('whitegrid')

## 2. Análise Exploratória (EDA)

A seguir, faremos uma análise de cada loja em relação ao faturamento, stisfação do cliente, custo de frete e análise geográfica

### 2.1  Faturamento por Loja

Para o faturamento por loja, somaremos os falores da coluna 'Preço', onde separaremos pela coluna 'Loja'

In [ ]:
faturamento = df.groupby('Loja')['Preço'].sum().sort_values(ascending=False).reset_index()

print(faturamento)

In [ ]:
# Plotando o gráfico do faturamento
plt.figure(figsize=(10, 6))
sns.barplot(data=faturamento, x='Loja', y='Preço', hue='Loja', palette='Blues_d', legend=False)
plt.title('Faturamento Total por Loja', pad=20)
plt.ylabel('Faturamento Total Por Loja')
plt.xlabel('')
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'R${x:,.0f}'))
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

Mesmo que a loja 1 possua um faturamento total maior vemos que todas as lojas possuem faturamentos similares. Logo, para uma melhro acurácia, analisaremos as vendas ao logo dos anos

In [ ]:
# Extrair ano da data
df['Ano'] = pd.to_datetime(df['Data da Compra'],  format='%d/%m/%Y').dt.year

# Calcular faturamento anual por loja
faturamento_anual = df.groupby(['Ano', 'Loja'])['Preço'].sum().unstack()
print(faturamento_anual)

In [ ]:
# Plotando gráfico do faturamento anual
plt.figure(figsize=(12, 6))
sns.barplot(data=df, x='Loja', y='Preço', hue='Ano', estimator=sum,
            palette='viridis_d', errorbar=None)
plt.title('Evolução Anual do Faturamento por Loja', pad=20)
plt.xlabel('Lojas')
plt.ylabel('Faturamento (R$)')
plt.yticks(range(0, 550000, 50000))
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'R${x:,.0f}'))
plt.legend(title='Ano', bbox_to_anchor=(1.05, 1))
plt.tight_layout()
plt.show()

Despresando o ano de 2023, que não estava complento, vemos uma queda nos faturamentos anuais na loja 1 e um aumento nas vendas anuais da loja 2. Nota-se uma variação na Loja 3 e na Loja 4

### 2.2. Desempenho por Categoria  

Cada loja pode ter categorias mais procuradas, portanto é importante analisar quais são os pontos fortes de cada setor, pela **quantidade** de produtos vendidos em cada categoria por loja.

In [ ]:
# Calcula a quantidade de vendas por categoria em cada loja
vendas_categoria_volume = df.groupby(['Loja', 'Categoria do Produto']).size().reset_index(name='Qtd_Vendas')

# Ordena por loja e quantidade de vendas
vendas_categoria_volume = vendas_categoria_volume.sort_values(['Loja', 'Qtd_Vendas'], ascending=[True, False])

print("Volume de Vendas por Categoria:\n")

lojas = vendas_categoria_volume['Loja'].unique()

for loja in lojas:
    tabela_loja = vendas_categoria_volume[vendas_categoria_volume['Loja'] == loja]
    display(tabela_loja.style.background_gradient(cmap='Greens', subset=['Qtd_Vendas']))

In [ ]:
# Criar uma coluna com a ordem de popularidade de categoria para cada loja
vendas_categoria_volume['Rank'] = vendas_categoria_volume.groupby('Loja')['Qtd_Vendas'].rank(ascending=False, method='dense')

# Plot dos gráficos
plt.figure(figsize=(14, 10))
g = sns.FacetGrid(
    vendas_categoria_volume,
    col='Loja',
    col_wrap=2,
    height=5,
    sharex=False,
    sharey=False,
    aspect=1.3
)

# Função para plotar cada subgráfico com ordem própria
# Calcula o total de vendas por loja
totais_por_loja = vendas_categoria_volume.groupby('Loja')['Qtd_Vendas'].sum()

# Função para plotar cada subgráfico com ordenação própria e total
def plot_ordered_bars(data, **kwargs):
    loja = data['Loja'].iloc[0]
    ordered_categories = data.sort_values('Qtd_Vendas', ascending=False)['Categoria do Produto']

    ax = sns.barplot(
        data=data,
        x='Qtd_Vendas',
        y='Categoria do Produto',
        hue='Categoria do Produto',
        legend=False,
        order=ordered_categories,
        palette='viridis',
        **kwargs
    )

    # Adiciona os valores nas barras
    for p in ax.patches:
        width = p.get_width()
        if width > 0:
            ax.text(
                width + 0.5,
                p.get_y() + p.get_height()/2.,
                f'{int(width)}',
                ha='left',
                va='center',
                fontsize=9
            )

    # Adiciona a legenda com o total de vendas da loja no canto superior direito
    total_vendas = totais_por_loja[loja]
    ax.text(
        0.95, -0.1,
        f'Total de Vendas: {int(total_vendas)}',
        transform=ax.transAxes,
        fontsize=11,
        fontweight='bold',
        ha='right',
        va='top',
        bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='gray', alpha=0.8)
    )

# Aplicar a função personalizada
g.map_dataframe(plot_ordered_bars)

# Configurações estéticas
g.set_titles('{col_name}', size=14, pad=10)
g.set_axis_labels('Número de Vendas', '')
g.fig.suptitle('Volume de Vendas por Categoria em Cada Loja', y=1.02, size=16)
plt.tight_layout()
plt.show()

Ao observar como cada loja se distribue nas categorias de produtos, vemos que todas as lojas são parecidas em relaçãs as categorias mais populares, com 'Móveis', 'Eletrônicos e 'Brinquedos' no top três. Com isso, podemos levar em conta outros fotores para a análise.

### 2.3. Satisfação do Cliente  

Um ponto deveras importante, é analisar se os clientes estão satisfeitos com o atendimento e as compras feitas nas lojas. Para isso, faremos um média de avaliação por loja.

In [ ]:
avaliacao_media = df.groupby('Loja')['Avaliação da compra'].mean().sort_values(ascending=False)
print(avaliacao_media)

In [ ]:
avaliacao_media.plot(kind='barh', color='green', title='Média de Avaliação por Loja')


As médias das lojas são bem similares, então para uma análise mais profudas poderíamos levar em consideração a outros dados estatísticos como moda e mediana, além de quantidade de avaliações ruins (1-2) médias (

## 2.4. Produtos mais e menos vendidos

Outra informação válida são os produtos preferidos pelos clientes de cada loja

In [ ]:
top_produtos = df.groupby(['Loja', 'Produto']).size().groupby('Loja').nlargest(5)

print(top_produtos)


### 2.5. Frete  médio por loja

In [ ]:
frete_medio = df.groupby('Loja')['Frete'].mean().sort_values()
frete_medio.plot(kind='pie', autopct='%.1f%%', title='Frete Médio por Loja')

### 2.6. Análise Geográfica 

## 3. Insights Críticos  


## 4. Relatório final